In [25]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np

# Exploring
import scipy.stats as stats
import sklearn.impute

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

# change default jupyter notebook column display
pd.set_option('display.max_columns', None)

import acquire

# Acquire (acquire.py)

### Zillow

> For the following, iterate through the steps you would take to create functions: Write the code to do the following in a jupyter notebook, test it, convert to functions, then create the file to house those functions.

> You will have a zillow.ipynb file and a helper file for each section in the pipeline.

#### **acquire & summarize**

1. Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

    - Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for `airconditioningtypeid`.

    - Only include properties with a transaction in 2017, and include only the last transaction for each properity (so no duplicate property ID's), along with zestimate error and date of transaction.

    - Only include properties that include a latitude and longitude value.


2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)


3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.


4. Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

In [6]:
df = acquire.wrangle_zillow()

In [7]:
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,1727539,14297519,nan,nan,nan,3.50,4.00,nan,nan,3.50,nan,nan,"3,100.00","3,100.00",nan,nan,nan,nan,"6,059.00",nan,3.00,2.00,633.00,nan,nan,"33,634,931.00","-117,869,207.00","4,506.00",nan,nan,nan,nan,nan,122,261.00,None,"60,590,630.07","53,571.00","1,286.00",nan,"96,978.00",0.00,nan,1.00,nan,nan,nan,nan,"1,998.00",nan,nan,"485,713.00","1,023,282.00","2,016.00","537,569.00","11,013.72",None,nan,"60,590,630,072,012.00",0.03,2017-01-01,None,None,None,None,Single Family Residential,None,None
1,1387261,17052889,nan,nan,nan,1.00,2.00,nan,nan,1.00,nan,"1,465.00","1,465.00","1,465.00",nan,nan,"1,465.00",nan,"6,111.00",1.00,1.00,1.00,0.00,nan,nan,"34,449,266.00","-119,281,531.00","12,647.00",nan,nan,nan,nan,nan,1110,261.00,None,"61,110,010.02","13,091.00","2,061.00",nan,"97,099.00",5.00,nan,nan,nan,nan,nan,nan,"1,967.00",1.00,nan,"88,000.00","464,000.00","2,016.00","376,000.00","5,672.48",None,nan,"61,110,010,023,006.00",0.06,2017-01-01,None,None,None,None,Single Family Residential,None,None
2,11677,14186244,nan,nan,nan,2.00,3.00,nan,nan,2.00,nan,nan,"1,243.00","1,243.00",nan,nan,nan,nan,"6,059.00",nan,2.00,2.00,440.00,nan,nan,"33,886,168.00","-117,823,170.00","8,432.00",1.00,nan,nan,nan,1.00,122,261.00,None,"60,590,218.02","21,412.00","1,286.00",nan,"97,078.00",6.00,nan,nan,nan,nan,nan,nan,"1,962.00",1.00,nan,"85,289.00","564,778.00","2,016.00","479,489.00","6,488.30",None,nan,"60,590,218,022,012.00",0.01,2017-01-01,None,None,None,None,Single Family Residential,None,None
3,2288172,12177905,nan,nan,nan,3.00,4.00,nan,8.00,3.00,nan,nan,"2,376.00","2,376.00",nan,nan,nan,nan,"6,037.00",nan,3.00,nan,nan,nan,2.00,"34,245,180.00","-118,240,722.00","13,038.00",1.00,nan,nan,nan,1.00,0101,261.00,LCR110000*,"60,373,001.00","396,551.00","3,101.00",nan,"96,330.00",0.00,nan,nan,nan,1.00,nan,nan,"1,970.00",nan,nan,"108,918.00","145,143.00","2,016.00","36,225.00","1,777.51",None,nan,"60,373,001,001,006.00",-0.10,2017-01-01,None,None,None,Central,Single Family Residential,None,None
4,1970746,10887214,1.00,nan,nan,3.00,3.00,nan,8.00,3.00,nan,nan,"1,312.00","1,312.00",nan,nan,nan,nan,"6,037.00",nan,3.00,nan,nan,nan,2.00,"34,185,120.00","-118,414,640.00","278,581.00",1.00,nan,nan,nan,1.00,010C,266.00,LAR3,"60,371,236.01","12,447.00","3,101.00","268,548.00","96,451.00",0.00,nan,nan,nan,1.00,nan,nan,"1,964.00",nan,nan,"73,681.00","119,407.00","2,016.00","45,726.00","1,533.89",None,nan,"60,371,236,012,000.00",0.01,2017-01-01,Central,None,None,Central,Condominium,None,None


In [8]:
df = pd.concat([
    df.iloc[:, :11], # all the rows, and up to, but not including the index of the column to drop
    df.iloc[:, 11 + 1:] # all the rows, and everything after the column to drop
], axis=1)

In [9]:
#sort values by parcelid and transactiondate
df = df.sort_values(["parcelid", "transactiondate"])

# drop duplicates
df = df.drop_duplicates(subset=["parcelid"],keep='last')

In [10]:
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
54793,1087254,10711855,nan,nan,nan,2.00,3.00,nan,8.00,2.00,nan,"2,107.00","2,107.00",nan,nan,nan,nan,"6,037.00",nan,2.00,nan,nan,nan,2.00,"34,222,559.00","-118,617,387.00","9,158.00",1.00,nan,nan,nan,1.00,0101,261.00,LARE9,"60,371,132.32","12,447.00","3,101.00","268,588.00","96,339.00",0.00,nan,nan,nan,1.00,nan,nan,"1,972.00",nan,nan,"249,655.00","624,139.00","2,016.00","374,484.00","7,659.36",None,nan,"60,371,132,321,007.00",-0.01,2017-07-07,None,None,None,Central,Single Family Residential,None,None
71153,1072280,10711877,1.00,nan,nan,2.00,4.00,nan,8.00,2.00,nan,"1,882.00","1,882.00",nan,nan,nan,nan,"6,037.00",nan,2.00,nan,nan,nan,2.00,"34,220,261.00","-118,616,409.00","9,035.00",1.00,nan,nan,nan,1.00,0101,261.00,LARE9,"60,371,132.32","12,447.00","3,101.00","268,588.00","96,339.00",0.00,nan,nan,nan,1.00,nan,nan,"1,972.00",nan,nan,"253,000.00","660,000.00","2,016.00","407,000.00","8,123.91",None,nan,"60,371,132,321,007.00",0.02,2017-08-29,Central,None,None,Central,Single Family Residential,None,None
23080,1340933,10711888,1.00,nan,nan,2.00,4.00,nan,8.00,2.00,nan,"1,882.00","1,882.00",nan,nan,nan,nan,"6,037.00",nan,2.00,nan,nan,nan,2.00,"34,222,491.00","-118,616,854.00","9,800.00",nan,nan,nan,nan,nan,0100,261.00,LARE9,"60,371,132.32","12,447.00","3,101.00","268,588.00","96,339.00",0.00,nan,nan,nan,1.00,nan,nan,"1,972.00",nan,nan,"257,591.00","542,923.00","2,016.00","285,332.00","6,673.24",None,nan,"60,371,132,321,007.00",0.08,2017-04-04,Central,None,None,Central,Single Family Residential,None,None
17907,1878109,10711910,nan,nan,nan,2.00,3.00,nan,8.00,2.00,nan,"1,477.00","1,477.00",nan,nan,nan,nan,"6,037.00",nan,2.00,nan,nan,nan,2.00,"34,221,864.00","-118,615,739.00","11,285.00",1.00,nan,nan,nan,1.00,0101,261.00,LARE11,"60,371,132.32","12,447.00","3,101.00","268,588.00","96,339.00",0.00,nan,nan,nan,1.00,nan,nan,"1,960.00",nan,nan,"57,968.00","78,031.00","2,016.00","20,063.00","1,116.46",None,nan,"60,371,132,321,008.00",-0.04,2017-03-17,None,None,None,Central,Single Family Residential,None,None
20258,2190858,10711923,nan,nan,nan,2.00,4.00,nan,8.00,2.00,nan,"1,918.00","1,918.00",nan,nan,nan,nan,"6,037.00",nan,2.00,nan,nan,nan,2.00,"34,220,619.00","-118,615,253.00","11,239.00",1.00,nan,nan,nan,1.00,0101,261.00,LARE11,"60,371,132.32","12,447.00","3,101.00","268,588.00","96,339.00",0.00,nan,nan,nan,1.00,nan,nan,"1,960.00",nan,nan,"167,869.00","415,459.00","2,016.00","247,590.00","5,239.85",None,nan,"60,371,132,321,008.00",-0.01,2017-03-24,None,None,None,Central,Single Family Residential,None,None


### Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)¶


In [11]:
df.shape

(77381, 67)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77381 entries, 54793 to 16553
Data columns (total 67 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            77381 non-null  int64  
 1   parcelid                      77381 non-null  int64  
 2   airconditioningtypeid         24953 non-null  float64
 3   architecturalstyletypeid      206 non-null    float64
 4   basementsqft                  50 non-null     float64
 5   bathroomcnt                   77381 non-null  float64
 6   bedroomcnt                    77381 non-null  float64
 7   buildingclasstypeid           15 non-null     float64
 8   buildingqualitytypeid         49672 non-null  float64
 9   calculatedbathnbr             76772 non-null  float64
 10  decktypeid                    614 non-null    float64
 11  calculatedfinishedsquarefeet  77185 non-null  float64
 12  finishedsquarefeet12          73749 non-null  float64
 1

In [13]:
df.dtypes

id                            int64
parcelid                      int64
airconditioningtypeid       float64
architecturalstyletypeid    float64
basementsqft                float64
                             ...   
buildingclassdesc            object
heatingorsystemdesc          object
propertylandusedesc          object
storydesc                    object
typeconstructiondesc         object
Length: 67, dtype: object

### Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.¶


In [14]:
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
54793,1087254,10711855,nan,nan,nan,2.00,3.00,nan,8.00,2.00,nan,"2,107.00","2,107.00",nan,nan,nan,nan,"6,037.00",nan,2.00,nan,nan,nan,2.00,"34,222,559.00","-118,617,387.00","9,158.00",1.00,nan,nan,nan,1.00,0101,261.00,LARE9,"60,371,132.32","12,447.00","3,101.00","268,588.00","96,339.00",0.00,nan,nan,nan,1.00,nan,nan,"1,972.00",nan,nan,"249,655.00","624,139.00","2,016.00","374,484.00","7,659.36",None,nan,"60,371,132,321,007.00",-0.01,2017-07-07,None,None,None,Central,Single Family Residential,None,None
71153,1072280,10711877,1.00,nan,nan,2.00,4.00,nan,8.00,2.00,nan,"1,882.00","1,882.00",nan,nan,nan,nan,"6,037.00",nan,2.00,nan,nan,nan,2.00,"34,220,261.00","-118,616,409.00","9,035.00",1.00,nan,nan,nan,1.00,0101,261.00,LARE9,"60,371,132.32","12,447.00","3,101.00","268,588.00","96,339.00",0.00,nan,nan,nan,1.00,nan,nan,"1,972.00",nan,nan,"253,000.00","660,000.00","2,016.00","407,000.00","8,123.91",None,nan,"60,371,132,321,007.00",0.02,2017-08-29,Central,None,None,Central,Single Family Residential,None,None
23080,1340933,10711888,1.00,nan,nan,2.00,4.00,nan,8.00,2.00,nan,"1,882.00","1,882.00",nan,nan,nan,nan,"6,037.00",nan,2.00,nan,nan,nan,2.00,"34,222,491.00","-118,616,854.00","9,800.00",nan,nan,nan,nan,nan,0100,261.00,LARE9,"60,371,132.32","12,447.00","3,101.00","268,588.00","96,339.00",0.00,nan,nan,nan,1.00,nan,nan,"1,972.00",nan,nan,"257,591.00","542,923.00","2,016.00","285,332.00","6,673.24",None,nan,"60,371,132,321,007.00",0.08,2017-04-04,Central,None,None,Central,Single Family Residential,None,None
17907,1878109,10711910,nan,nan,nan,2.00,3.00,nan,8.00,2.00,nan,"1,477.00","1,477.00",nan,nan,nan,nan,"6,037.00",nan,2.00,nan,nan,nan,2.00,"34,221,864.00","-118,615,739.00","11,285.00",1.00,nan,nan,nan,1.00,0101,261.00,LARE11,"60,371,132.32","12,447.00","3,101.00","268,588.00","96,339.00",0.00,nan,nan,nan,1.00,nan,nan,"1,960.00",nan,nan,"57,968.00","78,031.00","2,016.00","20,063.00","1,116.46",None,nan,"60,371,132,321,008.00",-0.04,2017-03-17,None,None,None,Central,Single Family Residential,None,None
20258,2190858,10711923,nan,nan,nan,2.00,4.00,nan,8.00,2.00,nan,"1,918.00","1,918.00",nan,nan,nan,nan,"6,037.00",nan,2.00,nan,nan,nan,2.00,"34,220,619.00","-118,615,253.00","11,239.00",1.00,nan,nan,nan,1.00,0101,261.00,LARE11,"60,371,132.32","12,447.00","3,101.00","268,588.00","96,339.00",0.00,nan,nan,nan,1.00,nan,nan,"1,960.00",nan,nan,"167,869.00","415,459.00","2,016.00","247,590.00","5,239.85",None,nan,"60,371,132,321,008.00",-0.01,2017-03-24,None,None,None,Central,Single Family Residential,None,None


In [15]:
def rows_missing(df):
    df1 = pd.DataFrame(df.isnull().sum(), columns = ['num_rows_missing'])
    df1['pct_rows_missing'] = df1.num_rows_missing/df.shape[0]
    return df1

In [16]:
df1 = rows_missing(df)
df1

,num_rows_missing,pct_rows_missing
id,0,0.00
parcelid,0,0.00
airconditioningtypeid,52428,0.68
architecturalstyletypeid,77175,1.00
basementsqft,77331,1.00
...,...,...
buildingclassdesc,77366,1.00
heatingorsystemdesc,27941,0.36
propertylandusedesc,0,0.00
storydesc,77331,1.00


#### Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.¶


In [17]:
def cols_missing(df):
    df['total'] = df.apply(lambda row: row.isnull().sum(), axis =1)
    df2 = df[['total']]
    df2 = df2.reset_index()
    df2 = df2.groupby('total').count()
    df2= df2.reset_index()
    df2.rename(columns = {'total': 'num_cols_missing', 'index': 'num_rows'}, inplace = True)
    df2['pct_cols_missing'] = df2.num_cols_missing/df.shape[1]
    return df2

In [18]:
df2 = cols_missing(df)
df2.head()

,num_cols_missing,num_rows,pct_cols_missing
0,22,2,0.32
1,23,11,0.34
2,24,23,0.35
3,25,55,0.37
4,26,252,0.38


### Prepare

Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.


In [19]:
def single_units(df):
    df = df[(df.propertylandusedesc=='Single Family Residential') & ((df.unitcnt<=1)|df.unitcnt.isnull()) & (df.bedroomcnt>0)\
           & (df.bathroomcnt>0) & (df.calculatedfinishedsquarefeet>350)]
    return df

In [20]:
df = single_units(df)

Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

The input: A dataframe\ A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).\ A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).


In [21]:
def handle_missing_values(df, prop_required_column = .5, prop_required_row = .75):
    threshold = int(round(prop_required_column*len(df.index),0))
    df.dropna(axis=1, thresh=threshold, inplace=True)
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df

Decide how to handle the remaining missing values:

Fill with constant value.\ Impute with mean, median, mode.\ Drop row/column

In [22]:
df = handle_missing_values(df)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52131 entries, 54793 to 35382
Data columns (total 35 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            52131 non-null  int64  
 1   parcelid                      52131 non-null  int64  
 2   bathroomcnt                   52131 non-null  float64
 3   bedroomcnt                    52131 non-null  float64
 4   buildingqualitytypeid         33603 non-null  float64
 5   calculatedbathnbr             52116 non-null  float64
 6   calculatedfinishedsquarefeet  52131 non-null  float64
 7   finishedsquarefeet12          51973 non-null  float64
 8   fips                          52131 non-null  float64
 9   fullbathcnt                   52116 non-null  float64
 10  heatingorsystemtypeid         33807 non-null  float64
 11  latitude                      52131 non-null  float64
 12  longitude                     52131 non-null  float64
 1

In [27]:
knn_imputer = sklearn.impute.KNNImputer(n_neighbors=3)

knn_imputer.fit(df.select_dtypes(exclude=[object]))

KNNImputer(add_indicator=False, copy=True, metric='nan_euclidean',
           missing_values=nan, n_neighbors=3, weights='uniform')

In [28]:
df3 = pd.DataFrame(knn_imputer.transform(df.select_dtypes(exclude=[object])), columns = df.select_dtypes(exclude='object').columns, index = df.index)

In [29]:
df3

,id,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,propertylandusetypeid,rawcensustractandblock,regionidcity,regionidcounty,regionidzip,roomcnt,unitcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,total
54793,"1,087,254.00","10,711,855.00",2.00,3.00,8.00,2.00,"2,107.00","2,107.00","6,037.00",2.00,2.00,"34,222,559.00","-118,617,387.00","9,158.00",261.00,"60,371,132.32","12,447.00","3,101.00","96,339.00",0.00,1.00,"1,972.00","249,655.00","624,139.00","2,016.00","374,484.00","7,659.36","60,371,132,321,007.00",-0.01,30.00
71153,"1,072,280.00","10,711,877.00",2.00,4.00,8.00,2.00,"1,882.00","1,882.00","6,037.00",2.00,2.00,"34,220,261.00","-118,616,409.00","9,035.00",261.00,"60,371,132.32","12,447.00","3,101.00","96,339.00",0.00,1.00,"1,972.00","253,000.00","660,000.00","2,016.00","407,000.00","8,123.91","60,371,132,321,007.00",0.02,28.00
23080,"1,340,933.00","10,711,888.00",2.00,4.00,8.00,2.00,"1,882.00","1,882.00","6,037.00",2.00,2.00,"34,222,491.00","-118,616,854.00","9,800.00",261.00,"60,371,132.32","12,447.00","3,101.00","96,339.00",0.00,1.00,"1,972.00","257,591.00","542,923.00","2,016.00","285,332.00","6,673.24","60,371,132,321,007.00",0.08,30.00
17907,"1,878,109.00","10,711,910.00",2.00,3.00,8.00,2.00,"1,477.00","1,477.00","6,037.00",2.00,2.00,"34,221,864.00","-118,615,739.00","11,285.00",261.00,"60,371,132.32","12,447.00","3,101.00","96,339.00",0.00,1.00,"1,960.00","57,968.00","78,031.00","2,016.00","20,063.00","1,116.46","60,371,132,321,008.00",-0.04,30.00
20258,"2,190,858.00","10,711,923.00",2.00,4.00,8.00,2.00,"1,918.00","1,918.00","6,037.00",2.00,2.00,"34,220,619.00","-118,615,253.00","11,239.00",261.00,"60,371,132.32","12,447.00","3,101.00","96,339.00",0.00,1.00,"1,960.00","167,869.00","415,459.00","2,016.00","247,590.00","5,239.85","60,371,132,321,008.00",-0.01,30.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54576,"2,684,284.00","167,636,727.00",3.00,4.00,4.00,3.00,"2,670.00","2,670.00","6,037.00",3.00,2.00,"34,681,736.00","-118,096,714.00","6,022.67",261.00,"60,379,005.06","24,934.33","3,101.00","97,103.00",0.00,1.00,"2,006.00","182,200.00","242,900.00","2,016.00","60,700.00","3,865.75","61,110,046,394,719.00",0.02,35.00
4161,"1,596,111.00","167,637,054.00",1.00,2.00,7.00,1.00,756.00,756.00,"6,037.00",1.00,7.00,"33,863,071.00","-118,379,246.00","6,670.33",261.00,"60,376,207.02","37,630.67","3,101.00","97,097.00",0.00,1.00,"1,953.00","20,083.00","84,683.00","2,016.00","64,600.00","1,631.15","61,110,048,054,688.66",-0.22,37.00
62351,"358,149.00","167,637,371.00",5.00,4.00,8.00,5.00,"4,000.00","4,000.00","6,037.00",5.00,2.00,"34,089,958.00","-118,451,987.00","6,670.33",261.00,"60,372,621.00","37,630.67","3,101.00","97,097.00",0.00,1.00,"2,013.00","1,664,376.00","2,597,367.00","2,016.00","932,991.00","54,391.29","61,110,048,054,688.66",2.07,34.00
45965,"835,451.00","167,638,003.00",3.00,3.00,4.00,3.00,"2,108.00","2,108.00","6,037.00",3.00,2.00,"34,126,076.00","-118,226,237.00","6,670.33",261.00,"60,371,861.00","37,630.67","3,101.00","97,097.00",0.00,1.00,"2,014.00","303,559.00","709,659.00","2,016.00","406,100.00","8,621.13","61,110,048,054,688.66",-0.16,35.00


In [30]:
pd.merge(df3, df[['transactiondate', 'propertycountylandusecode','propertylandusedesc']], left_index = True, right_index = True, how = 'left')

,id,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,propertylandusetypeid,rawcensustractandblock,regionidcity,regionidcounty,regionidzip,roomcnt,unitcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,total,transactiondate,propertycountylandusecode,propertylandusedesc
54793,"1,087,254.00","10,711,855.00",2.00,3.00,8.00,2.00,"2,107.00","2,107.00","6,037.00",2.00,2.00,"34,222,559.00","-118,617,387.00","9,158.00",261.00,"60,371,132.32","12,447.00","3,101.00","96,339.00",0.00,1.00,"1,972.00","249,655.00","624,139.00","2,016.00","374,484.00","7,659.36","60,371,132,321,007.00",-0.01,30.00,2017-07-07,0101,Single Family Residential
71153,"1,072,280.00","10,711,877.00",2.00,4.00,8.00,2.00,"1,882.00","1,882.00","6,037.00",2.00,2.00,"34,220,261.00","-118,616,409.00","9,035.00",261.00,"60,371,132.32","12,447.00","3,101.00","96,339.00",0.00,1.00,"1,972.00","253,000.00","660,000.00","2,016.00","407,000.00","8,123.91","60,371,132,321,007.00",0.02,28.00,2017-08-29,0101,Single Family Residential
23080,"1,340,933.00","10,711,888.00",2.00,4.00,8.00,2.00,"1,882.00","1,882.00","6,037.00",2.00,2.00,"34,222,491.00","-118,616,854.00","9,800.00",261.00,"60,371,132.32","12,447.00","3,101.00","96,339.00",0.00,1.00,"1,972.00","257,591.00","542,923.00","2,016.00","285,332.00","6,673.24","60,371,132,321,007.00",0.08,30.00,2017-04-04,0100,Single Family Residential
17907,"1,878,109.00","10,711,910.00",2.00,3.00,8.00,2.00,"1,477.00","1,477.00","6,037.00",2.00,2.00,"34,221,864.00","-118,615,739.00","11,285.00",261.00,"60,371,132.32","12,447.00","3,101.00","96,339.00",0.00,1.00,"1,960.00","57,968.00","78,031.00","2,016.00","20,063.00","1,116.46","60,371,132,321,008.00",-0.04,30.00,2017-03-17,0101,Single Family Residential
20258,"2,190,858.00","10,711,923.00",2.00,4.00,8.00,2.00,"1,918.00","1,918.00","6,037.00",2.00,2.00,"34,220,619.00","-118,615,253.00","11,239.00",261.00,"60,371,132.32","12,447.00","3,101.00","96,339.00",0.00,1.00,"1,960.00","167,869.00","415,459.00","2,016.00","247,590.00","5,239.85","60,371,132,321,008.00",-0.01,30.00,2017-03-24,0101,Single Family Residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54576,"2,684,284.00","167,636,727.00",3.00,4.00,4.00,3.00,"2,670.00","2,670.00","6,037.00",3.00,2.00,"34,681,736.00","-118,096,714.00","6,022.67",261.00,"60,379,005.06","24,934.33","3,101.00","97,103.00",0.00,1.00,"2,006.00","182,200.00","242,900.00","2,016.00","60,700.00","3,865.75","61,110,046,394,719.00",0.02,35.00,2017-07-07,0100,Single Family Residential
4161,"1,596,111.00","167,637,054.00",1.00,2.00,7.00,1.00,756.00,756.00,"6,037.00",1.00,7.00,"33,863,071.00","-118,379,246.00","6,670.33",261.00,"60,376,207.02","37,630.67","3,101.00","97,097.00",0.00,1.00,"1,953.00","20,083.00","84,683.00","2,016.00","64,600.00","1,631.15","61,110,048,054,688.66",-0.22,37.00,2017-01-19,0100,Single Family Residential
62351,"358,149.00","167,637,371.00",5.00,4.00,8.00,5.00,"4,000.00","4,000.00","6,037.00",5.00,2.00,"34,089,958.00","-118,451,987.00","6,670.33",261.00,"60,372,621.00","37,630.67","3,101.00","97,097.00",0.00,1.00,"2,013.00","1,664,376.00","2,597,367.00","2,016.00","932,991.00","54,391.29","61,110,048,054,688.66",2.07,34.00,2017-07-31,0101,Single Family Residential
45965,"835,451.00","167,638,003.00",3.00,3.00,4.00,3.00,"2,108.00","2,108.00","6,037.00",3.00,2.00,"34,126,076.00","-118,226,237.00","6,670.33",261.00,"60,371,861.00","37,630.67","3,101.00","97,097.00",0.00,1.00,"2,014.00","303,559.00","709,659.00","2,016.00","406,100.00","8,621.13","61,110,048,054,688.66",-0.16,35.00,2017-06-14,0100,Single Family Residential


In [31]:
print(sklearn.__version__)

0.22.2.post1
